<a href="https://colab.research.google.com/github/Takily/dbn-vs-cnn-image-classification/blob/main/Propaganda_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
with zipfile.ZipFile("ka_1.zip", "r") as z:
    z.extractall(".")
!ls


ka_1  ka_1.zip	__MACOSX  sample_data


In [2]:
import csv, os, re

In [5]:
# Paths in your Colab working directory
ARTICLES_DIR = "ka_1/test-articles-subtask-3"
LABELS_FILE  = "ka_1/test-labels-subtask-3.txt"
OUTPUT_CSV   = "ka_eval_paragraphs.csv"
INCLUDE_UNLABELED = False  # True if you also want paragraphs with no labels


In [6]:
# 1) Read all article texts and split into paragraphs
articles = {}
for fname in sorted(os.listdir(ARTICLES_DIR)):
    if fname.startswith("article") and fname.endswith(".txt"):
        article_id = re.findall(r"\d+", fname)[0]
        with open(os.path.join(ARTICLES_DIR, fname), "r", encoding="utf-8") as f:
            text = f.read()
        paragraphs = [p.strip() for p in re.split(r"\n{1,}", text) if p.strip()]
        articles[article_id] = paragraphs
print(f"Loaded {len(articles)} articles")

Loaded 29 articles


In [7]:
# 2) Read paragraph-level labels
labels_map = {}
with open(LABELS_FILE, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line: continue
        parts = line.split()
        if len(parts) < 3: continue
        art_id = parts[0]
        para_idx = int(parts[1]) - 1  # 0-based
        labels_str = " ".join(parts[2:])
        labs = [l.strip() for l in re.split(r"[,\s]+", labels_str) if l.strip()]
        labels_map[(art_id, para_idx)] = labs

In [8]:
rows = []
for art_id, paras in articles.items():
    for i, text in enumerate(paras):
        key = (art_id, i)
        labs = labels_map.get(key, [])
        if labs or INCLUDE_UNLABELED:
            rows.append({
                "article_id": art_id,
                "para_id": i + 1,
                "text": text,
                "labels": labs
            })
print(f"Prepared {len(rows)} paragraph rows")

Prepared 38 paragraph rows


In [12]:
# 4) Write CSV
with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["text", "labels", "article_id", "para_id"])
    writer.writeheader()
    for r in rows:
        writer.writerow({
            "text": r["text"],
            "labels": ",".join(r["labels"]),
            "article_id": r["article_id"],
            "para_id": r["para_id"]
        })
print(f"Saved to {OUTPUT_CSV}")

Saved to ka_eval_paragraphs.csv


In [15]:
!python convert_paragraphs_to_csv.py

python3: can't open file '/content/convert_paragraphs_to_csv.py': [Errno 2] No such file or directory


In [ ]:
# === Install once (safe to re-run) ===
!pip install -q transformers accelerate torch scikit-learn

# === Imports ===
import csv, json, re, sys
import numpy as np
from typing import List
from sklearn.metrics import precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

# === Config ===
CSV_PATH   = "ka_eval_paragraphs.csv"             # your prepared eval CSV
MODEL_NAME = "ai-forever/mGPT"                    # switch to "ai-forever/mGPT-1.3B-georgian" to use the Georgian-only model
USE_FEW_SHOT = False                              # set True to enable the small FEW_SHOTS block

# Few-shot seeds (tiny, editable). Keep short. Only used if USE_FEW_SHOT=True.
FEW_SHOTS = [
    {"text": "ოპონენტები ხალხს ატყუებენ და საკუთარი ინტერესებისთვის იბრძვიან.",
     "labels_en": ["Loaded_Language","Name_Calling-Labeling"]},
    {"text": "ჩვენი ტრადიციები საფრთხეშია, ამიტომ უნდა ვიდგეთ ერთ მუშტად.",
     "labels_en": ["Flag_Waving","Appeal_to_Fear-Prejudice"]},
]

# Gold/eval label set (English — must match test set)
ALLOWED_LABELS_EN = [
    "Doubt", "Guilt_by_Association", "Exaggeration-Minimisation", "Loaded_Language",
    "Name_Calling-Labeling", "Questioning_the_Reputation", "Obfuscation-Vagueness-Confusion",
    "Flag_Waving", "Red_Herring", "Appeal_to_Hypocrisy", "Consequential_Oversimplification",
    "Causal_Oversimplification", "Repetition", "False_Dilemma-No_Choice", "Conversation_Killer",
    "Appeal_to_Fear-Prejudice", "Appeal_to_Values",
    "Appeal_to_Authority","Straw_Man"

]

# Georgian surface forms -> English mapping (expand if you want synonyms)
GEO2EN = {
    "ეჭვის აღძვრა": "Doubt",
    "დაბარელება ასოციაციით": "Guilt_by_Association",
    "გაზვიადება - დაკნინება": "Exaggeration-Minimisation",
    "პათეტიკური ენა": "Loaded_Language",
    "სახელების დაძახება": "Name_Calling-Labeling",
    "რეპუტაციის ეჭვქვეშ დაყენება": "Questioning_the_Reputation",
    "ბუნდოვანება": "Obfuscation-Vagueness-Confusion",
    "პოპულისტური პატრიოტიზმი": "Flag_Waving",
    "ყურადღების გადატანა": "Red_Herring",
    "თვალთმაქცობის დაბრალება": "Appeal_to_Hypocrisy",
    "გამარტივებული მიზეზშედეგობრიობა": "Consequential_Oversimplification",
    "მიზეზთა გამარტივება": "Causal_Oversimplification",
    "განმეორებითი რიტორიკა": "Repetition",
    "ცრუ დილემა - არჩევანის არარსებობა": "False_Dilemma-No_Choice",
    "დისკუსიის დამხშობი": "Conversation_Killer",
    "შიშით/წინასწარგანწყობით აპელირება": "Appeal_to_Fear-Prejudice",
    "ღირებულებებით აპელირება":"Appeal_to_Values",
    "ავტორიტეტით აპელირება":"Appeal_to_Authority",
    "ცრუ კონტრარგუმენტი":"Straw_Man"
 }

# Bilingual label list for the prompt (EN with KA gloss so the model has semantics)
BILINGUAL = [
    "Doubt (ეჭვის აღძვრა - სანდოობის შერყევა)",
    "Guilt_by_Association (დაბარელება ასოციაციით - სახელშელახულ ჯგუფთან გაიგივება)",
    "Exaggeration-Minimisation (გაზვიადება - დაკნინება - რეალობის გაბუქება ან შემცირება)",
    "Loaded_Language (პათეტიკური ენა - ემოციურად დატვირთული ენა)",
    "Name_Calling-Labeling (სახელების დაძახება - პოზიტიური ან ნეგატიური იარლიყით მანიპულაცია)",
    "Questioning_the_Reputation (რეპუტაციის ეჭვქვეშ დაყენება - მორალურ იმიჯზე დარტყმა)",
    "Obfuscation-Vagueness-Confusion (ბუნდოვანება - მიზანმიმართული,დამაბნეველი ორაზროვნება)",
    "Flag_Waving (პოპულისტური პატრიოტიზმი - მანიპულაცია ეროვნული პათოსით)",
    "Red_Herring (ყურადღების გადატანა - არამთავარი თემის შემოტყუება)",
    "Appeal_to_Hypocrisy (თვალთმაქცობის დაბრალება - ხაზგასმა ორმაგ სტანდარტზე)",
    "Consequential_Oversimplification (გამარტივებული მიზეზშედეგობრიობა - მანიპულირებული მოვლენათა ჯაჭვი)",
    "Causal_Oversimplification (მიზეზთა გამარტივება - მრავალფაქტორული პრობლემის გამარტივება)",
    "Repetition (განმეორებითი რიტორიკა - დაჟინებული ფრაზის ბრუნვა)",
    "False_Dilemma-No_Choice (ცრუ დილემა - არჩევანის არარსებობის ილუზიის შექმნა)",
    "Conversation_Killer (დისკუსიის დამხშობი - ნიუანსური დიალოგის ჩახშობა)",
    "Appeal_to_Fear-Prejudice (შიშით/წინასწარგანწყობით აპელირება - საფთხის გაზვიადებით ზემოქმედება)",
    "Appeal_to_Values (ღირებულებებით აპელირება - კეთილშობილურ ფასეულობებზე პარაზიტირება)",
    "Appeal_to_Authority (ავტორიტეტით აპელირება - )",
    "Straw_Man (ცრუ კონტრარგუმენტი - რეალური თემის გვერდის ავლა)"

]

# === Prompt builder (no JSON; zero-shot by default; optional few-shot) ===
def build_prompt(text: str) -> str:
    intro = (
        "ქვემოთ მოცემული ტექსტისთვის მიუთითე რომელი პროპაგანდის ტექნიკებია გამოყენებული.\n"
        "შეგიძლია უპასუხო ინგლისური ლეიბლებით ან ქვემოთ მოცემული ქართული აღწერებით.\n"
        "დაასახელე მხოლოდ შესაბამისი ტექნიკების სახელები, მძიმით გამოყოფილი. "
        "თუ არცერთი არ არის, შეგიძლია დაწერო: None.\n\n"
        "დასაშვები ტექნიკები:\n- " + "\n- ".join(BILINGUAL) + "\n\n"
    )
    few = ""
    if USE_FEW_SHOT and FEW_SHOTS:
        for ex in FEW_SHOTS:
            labs = ", ".join(ex["labels_en"])
            few += f"მაგალითი:\nტექსტი: {ex['text']}\nტექნიკები: {labs}\n\n"
    return (
        intro + few +
        "ტექსტი:\n<<<" + text.strip() + ">>>\n\n"
        "ტექნიკები: "
    )

# === Parsers ===
def gold_to_list(s: str) -> List[str]:
    s = (s or "").strip()
    if not s: return []
    if s.startswith("["):
        try:
            arr = json.loads(s)
            return [x for x in arr if x in ALLOWED_LABELS_EN]
        except Exception:
            pass
    parts = re.split(r"[,\|;]\s*", s)
    return [p for p in parts if p in ALLOWED_LABELS_EN]

def parse_prediction(raw: str) -> List[str]:
    """
    Accept either EN labels directly or KA forms mapped to EN.
    We look only at the first line to avoid rambling.
    """
    line = raw.strip().splitlines()[0] if raw.strip() else ""
    if "ტექნიკები" in line.lower():
        line = line.split(":", 1)[-1].strip()
    # split candidates
    parts = [p.strip() for p in re.split(r"[,\|;]+", line) if p.strip()]
    out = []
    for p in parts:
        if p in ALLOWED_LABELS_EN:
            out.append(p)
            continue
        # exact KA match
        if p in GEO2EN:
            out.append(GEO2EN[p])
            continue
        # fuzzy KA contains
        matched = False
        for ka, en in GEO2EN.items():
            if ka in p:
                out.append(en); matched = True; break
        if matched: continue
        # tolerate accidental spaces/typos around EN labels
        for en in ALLOWED_LABELS_EN:
            if en.replace("_"," ").lower() == p.replace("_"," ").lower():
                out.append(en); break
    # dedupe, keep order, filter
    seen = set()
    clean = [x for x in out if (x in ALLOWED_LABELS_EN and (x not in seen and not seen.add(x)))]
    if len(clean) == 1 and clean[0].lower() == "none":
        return []
    return clean

def binarize(batch: List[List[str]]):
    idx = {lab:i for i,lab in enumerate(ALLOWED_LABELS_EN)}
    Y = np.zeros((len(batch), len(ALLOWED_LABELS_EN)), dtype=int)
    for r, labs in enumerate(batch):
        for lab in labs:
            if lab in idx:
                Y[r, idx[lab]] = 1
    return Y

# === Load data ===
rows = []
with open(CSV_PATH, newline="", encoding="utf-8") as f:
    for r in csv.DictReader(f):
        rows.append({"text": r["text"], "gold": gold_to_list(r["labels"])})
print(f"Loaded {len(rows)} rows from {CSV_PATH}")
assert rows, "CSV is empty or path is wrong."

# === Load model ===
try:
    tok = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype="auto", device_map="auto")
except Exception as e:
    print("Falling back to CPU due to loading error:", e)
    tok = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="cpu")

cfg = GenerationConfig(max_new_tokens=64, temperature=0.7, top_p=0.95, do_sample=True)

# === Inference ===
preds, golds = [], []
for i, r in enumerate(rows, 1):
    prompt = build_prompt(r["text"])
    inputs = tok(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out = model.generate(**inputs, generation_config=cfg)
    raw = tok.decode(out[0][inputs["input_ids"].shape[1]:], skip_special_tokens=True)
    plabels = parse_prediction(raw)
    preds.append(plabels); golds.append(r["gold"])

    if i <= 5:
        print(f"\n--- EXAMPLE {i} ---")
        print("RAW:", raw[:400].replace("\n"," "))
        print("PRED:", plabels)
        print("GOLD:", r["gold"])

# === Metrics ===
Y_true, Y_pred = binarize(golds), binarize(preds)
micro = precision_recall_fscore_support(Y_true, Y_pred, average="micro", zero_division=0)
macro = precision_recall_fscore_support(Y_true, Y_pred, average="macro", zero_division=0)

print("\n=== ZERO-SHOT (no JSON) RESULTS ===")
print(f"Micro P/R/F1: {micro[0]:.3f}/{micro[1]:.3f}/{micro[2]:.3f}")
print(f"Macro P/R/F1: {macro[0]:.3f}/{macro[1]:.3f}/{macro[2]:.3f}")

# Per-label (optional)
per = precision_recall_fscore_support(Y_true, Y_pred, average=None, zero_division=0)
print("\nPer-label F1:")
for lab, f1 in zip(ALLOWED_LABELS_EN, per[2]):
    print(f"{lab}: {f1:.3f}")
